# Exercise 04 : A/B-testing

## Import

In [16]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library *sqlite3*

In [17]:
db_connect = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')

## Create two dataframes: `test_results` and `control_results`

In [18]:
sql_query="""

SELECT
    'before' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) < julianday(test.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) > julianday(test.first_view_ts)
"""

test_results = pd.read_sql(sql=sql_query, con=db_connect)
test_results

,time,avg_diff
0,after,-103.40625
1,before,-60.56250


In [19]:
sql_query="""

SELECT
    'before' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) < julianday(control.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) > julianday(control.first_view_ts)
"""

control_results = pd.read_sql(sql=sql_query, con=db_connect)
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286


In [20]:
percent_test = round((test_results.avg_diff[0] - test_results.avg_diff[1]) / test_results.avg_diff[0] * 100, 2)
print(percent_test, '%')

41.43 %


In [21]:
percent_control = round((control_results.avg_diff[0] - control_results.avg_diff[1]) / control_results.avg_diff[0] * 100, 2)
print(percent_control, '%')

11.75 %


## Close the connection

In [22]:
db_connect.close()